dict_keys(['Module'])


In [110]:
tree["Module"].keys()

dict_keys(['body', 'type_ignores'])

In [111]:
body = tree["Module"]["body"]
len(body)

5

In [112]:
body[0].keys()

dict_keys(['ImportFrom'])

In [113]:
body[1].keys()

dict_keys(['Import'])

In [114]:
body[2].keys()

dict_keys(['ImportFrom'])

In [115]:
body[3].keys()

dict_keys(['FunctionDef'])

In [116]:
body[3]['FunctionDef']['body']

[{'Expr': {'value': {'Call': {'func': {'Attribute': {'value': {'Name': {'id': 'config',
         'ctx': 'Load'}},
       'attr': 'load_kube_config',
       'ctx': 'Load'}},
     'args': [],
     'keywords': []}}}},
 {'FunctionDef': {'name': 'today',
   'args': {'arguments': {'posonlyargs': [],
     'args': [],
     'vararg': None,
     'kwonlyargs': [],
     'kw_defaults': [],
     'kwarg': None,
     'defaults': []}},
   'body': [{'Pass': {}}],
   'decorator_list': [],
   'returns': None,
   'type_comment': None}},
 {'With': {'items': [{'withitem': {'context_expr': {'Call': {'func': {'Name': {'id': 'open',
          'ctx': 'Load'}},
        'args': [{'Call': {'func': {'Attribute': {'value': {'Name': {'id': 'path',
               'ctx': 'Load'}},
             'attr': 'join',
             'ctx': 'Load'}},
           'args': [{'Call': {'func': {'Attribute': {'value': {'Name': {'id': 'path',
                  'ctx': 'Load'}},
                'attr': 'dirname',
                'ctx': 'Load

In [26]:
body[4].keys()

dict_keys(['If'])

In [104]:
body[4]['If']

{'test': {'Compare': {'left': {'Name': {'id': '__name__', 'ctx': 'Load'}},
   'ops': [{'Eq': {}}],
   'comparators': [{'Constant': {'value': '__main__', 'kind': None}}]}},
 'body': [{'Expr': {'value': {'Call': {'func': {'Name': {'id': 'main',
        'ctx': 'Load'}},
      'args': [],
      'keywords': []}}}}],
 'orelse': []}

In [105]:
body[4]['If']['body']

[{'Expr': {'value': {'Call': {'func': {'Name': {'id': 'main', 'ctx': 'Load'}},
     'args': [],
     'keywords': []}}}}]

In [2]:
from file_map_engine.ast_engine import make_ast


test_path = 'test\python\examples\\deployment_create.py'

test_path = 'D:\Code\AC2\AC2\src\main\\test\\python\\doc\\source\\conf.py'

# test_path = 'test\\file.py'
code = open(test_path, 'r').read()

tree = make_ast(code)

print(tree.keys())

dict_keys(['Module'])


In [73]:
new_data = {}

def wrapper_get(tree, test_path):
    new_data.clear()
    module = tree['Module']
    objects = module['body']
    get_call_tree(objects, test_path)
    return new_data
    
def get_call_tree(objects, par):
    for i in objects:
        keys = list(i.keys())
        type_ = keys[0]
        if keys[0] == 'FunctionDef':
            print(i['FunctionDef']['name'], "defined in", par)
            retrieve_inner_functions(i)
        elif keys[0] == 'Expr':
#             print(i['Expr']['value']['Call']['func']['Name']['ctx'])
            retrieve_inner_expr(i, par)
        elif keys[0] == 'Assign':
#             print(i['Expr']['value']['Call']['func']['Name']['ctx'])
            retrieve_inner_assign(i, par)
        elif keys[0] == 'With':
#             print(i['Expr']['value']['Call']['func']['Name']['ctx'])
            retrieve_inner_with(i, par)
        elif keys[0] == 'ImportFrom':
#             print(i['Expr']['value']['Call']['func']['Name']['ctx'])
            retrieve_inner_imp_fro(i, par)
        elif keys[0] == 'Import':
#             print(i['Expr']['value']['Call']['func']['Name']['ctx'])
            retrieve_inner_imp(i, par)
        else:
            if 'body' in list(i[keys[0]].keys()):
                get_call_tree(i[keys[0]]['body'], par)
            
def retrieve_inner_imp_fro(obj, from_):
    
        
    from_2 = obj['ImportFrom']['module']
    
    if from_ in new_data.keys():
        new_data[from_].append((from_2, 'module'))
    else:
        new_data[from_] = [(from_2, 'module')]
        
    names = obj['ImportFrom']['names']
    for name in names:
        name_true = name['alias']['name']
        print(name_true,"from", from_2,"into", from_)
        if from_2 in new_data.keys():
            new_data[from_2].append((name_true, 'func'))
        else:
            new_data[from_2] = [(name_true, 'func')]
    

            
def retrieve_inner_imp(obj, from_):
    names = obj['Import']['names']
    for name in names:
        name_true = name['alias']['name']
        print(name_true,"into" , from_)
        if from_ in new_data.keys():
            new_data[from_].append((name_true, 'module'))
        else:
            new_data[from_] = [(name_true, 'module')]
            

def retrieve_inner_functions(obj):
    from_func = obj['FunctionDef']['name']
    body_func = obj['FunctionDef']['body']
    get_call_tree(body_func, from_func)


def retrieve_inner_expr(obj, from_):
    keys = list(obj.keys())
    if list(obj[keys[0]].keys())[0] == "value":
        new_obj = obj[keys[0]]['value']
        if list(new_obj.keys())[0] == "Call":
            new_obj = new_obj['Call']
            if list(new_obj.keys())[0] == "func":
                new_obj = new_obj['func']
                
                
                
                if list(new_obj.keys())[0] == "Attribute":
                    try:
                        from_2 = new_obj['Attribute']['value']['Name']['id']
                        func_name_ = obj[keys[0]]['value']['Call']['func']['Attribute']['attr']
                        print(func_name_, "from", from_2,"called in", from_)

#                         if from_ in new_data.keys():
#                             if (from_2, 'module') not in new_data[from_]:
#                                 new_data[from_].append((from_2, 'module'))
#                         else:
#                             new_data[from_] = [(from_2, 'module')]

                        if from_2 in new_data.keys():
                            if (func_name_, 'func') not in new_data[from_2]:
                                new_data[from_2].append((func_name_, 'func'))
                        else:
                            new_data[from_2] = [(func_name_, 'func')]
                    except:
                        print("\n\n",new_obj['Attribute']['value']['Attribute'].keys(), "\n\n")
                    
                    
                    
                elif list(new_obj.keys())[0] == "Name":
                    if from_ in new_data.keys():
                        if (new_obj['Name']['id'], 'func') not in new_data[from_]:
                            new_data[from_].append((new_obj['Name']['id'], 'func'))
                    else:
                        new_data[from_] = [(new_obj['Name']['id'], 'func')]
                    print(new_obj['Name']['id'], "called from", from_)
    
    
    
def retrieve_inner_assign(obj, from_):
    keys = list(obj.keys())    
    keys2 = list(obj[keys[0]]['value'].keys())
    new_obj = obj[keys[0]]['value']
    key_ = keys2[0]
    
    if key_ == 'Call':
        new_obj = new_obj['Call']
        if list(new_obj.keys())[0] == "func":
            new_obj = new_obj['func']
            
            
#             print(new_obj.keys())
            
            
            if list(new_obj.keys())[0] == "Attribute":
                try:
                    from_2 = new_obj['Attribute']['value']['Name']['id']
                    func_name_ = obj[keys[0]]['value']['Call']['func']['Attribute']['attr']
                    print(func_name_, "from", from_2,"called in", from_)

#                         if from_ in new_data.keys():
#                             if (from_2, 'module') not in new_data[from_]:
#                                 new_data[from_].append((from_2, 'module'))
#                         else:
#                             new_data[from_] = [(from_2, 'module')]

                    if from_2 in new_data.keys():
                        if (func_name_, 'func') not in new_data[from_2]:
                            new_data[from_2].append((func_name_, 'func'))
                    else:
                        new_data[from_2] = [(func_name_, 'func')]
                except:
                    print("\n\n",new_obj['Attribute']['value']['Attribute'].keys(), "\n\n")

            elif list(new_obj.keys())[0] == "Name":
                print(new_obj['Name']['id'], "called from", from_)
                
                if from_ in new_data.keys():
                    if (new_obj['Name']['id'], 'func') not in new_data[from_]:
                        new_data[from_].append((new_obj['Name']['id'], 'func'))
                else:
                    new_data[from_] = [(new_obj['Name']['id'], 'func')]

    


def retrieve_inner_with(obj, from_):
    keys = list(obj.keys())
#     print("\n\n",obj[keys[0]].keys(),"\n\n")
#     print("\n\n")
#     print(obj[keys[0]]['items'])
    print("\n\n")
    print(obj[keys[0]]['items'][0]['withitem'].keys())
    print("\n\n")
    
    file_name = obj[keys[0]]['items'][0]['withitem']['optional_vars']['Name']['id']
    print("\n\n")
    
#     print(obj[keys[0]]['items'][0]['withitem']['context_expr']['Call']['args'][0]['Constant']['value'])
    print("\n\n")
    
    if from_ in new_data.keys():
        if (file_name, 'file') not in new_data[from_]:
            new_data[from_].append((file_name, 'file'))
    else:
        new_data[from_] = [(file_name, 'file')]
    
    get_call_tree(obj[keys[0]]['body'], from_)
    

        
    


In [75]:
dat = wrapper_get(tree, test_path)

os into D:\Code\AC2\AC2\src\main\test\python\doc\source\conf.py
re into D:\Code\AC2\AC2\src\main\test\python\doc\source\conf.py
shutil into D:\Code\AC2\AC2\src\main\test\python\doc\source\conf.py
sys into D:\Code\AC2\AC2\src\main\test\python\doc\source\conf.py
AutoStructify from recommonmark.transform into D:\Code\AC2\AC2\src\main\test\python\doc\source\conf.py



dict_keys(['context_expr', 'optional_vars'])






../../README.md



readlines from r called in D:\Code\AC2\AC2\src\main\test\python\doc\source\conf.py
sub from re called in D:\Code\AC2\AC2\src\main\test\python\doc\source\conf.py
append from new_readme called in D:\Code\AC2\AC2\src\main\test\python\doc\source\conf.py



dict_keys(['context_expr', 'optional_vars'])






README.md



writelines from n called in D:\Code\AC2\AC2\src\main\test\python\doc\source\conf.py
copy from shutil called in D:\Code\AC2\AC2\src\main\test\python\doc\source\conf.py


 dict_keys(['value', 'attr', 'ctx']) 


setup defined in D:\Code\AC2\AC2\src\

In [76]:
dat

{'D:\\Code\\AC2\\AC2\\src\\main\\test\\python\\doc\\source\\conf.py': [('os',
   'module'),
  ('re', 'module'),
  ('shutil', 'module'),
  ('sys', 'module'),
  ('recommonmark.transform', 'module'),
  ('r', 'file'),
  ('n', 'file')],
 'recommonmark.transform': [('AutoStructify', 'func')],
 'r': [('readlines', 'func')],
 're': [('sub', 'func')],
 'new_readme': [('append', 'func')],
 'n': [('writelines', 'func')],
 'shutil': [('copy', 'func')],
 'app': [('add_config_value', 'func'), ('add_transform', 'func')]}